In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools
from IPython.display import Image
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob
%matplotlib inline
pd.set_option("display.precision", 2)

In [22]:
from processing_helper import extract_data
paths = [
    r'./data',
]

result_columns = ['mean', 'std', 'min', 'max', 'leafs', 'time', 'ratio', 'name']
results = {}
stations = np.array([])
for path in paths:
    all_files = sorted(glob.glob(path + "/*.csv"))
    
    for idx, filename in enumerate(all_files):
        data, station_map = extract_data(filename)
        
        grouped_data = data.groupby(['rental_place', 'return_place']).size().reset_index()
        grouped_data = grouped_data.rename(columns={
            'rental_place': 'Source',
            'return_place': 'Target',
            0: 'Weight'
        })
        grouped_data['Type'] = 'Directed'
        new_filename = (filename.split('_')[-1]).split('.')[0]
        results['networks/'+new_filename+'_edges.csv'] = grouped_data
        
        print(type(station_map))
        stations = np.concatenate((stations, station_map))

        
stations = np.unique(stations)
mapping_dict = {stations[i]: i for i in range(0, len(stations))}
stations = pd.DataFrame(stations, columns=['name'])
stations.index.rename('value')
stations.to_csv('networks/nodes.csv', index=True, index_label='value')

for filename, data in results.items():
    print(filename)

    for feature_name in ['Source', 'Target']:
        data[feature_name] = data[feature_name].map(mapping_dict)
        
    data['Source'] = pd.to_numeric(data['Source'], downcast='integer')
    data['Target'] = pd.to_numeric(data['Target'], downcast='integer')
    data = data.dropna()
    data.to_csv(filename, index=False)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
networks/2019-03_edges.csv
      Source  Target  Weight      Type
0          1       2       1  Directed
1          1       5       5  Directed
2          1      10       5  Directed
3          1      15       1  Directed
4          1      18       6  Directed
...      ...     ...     ...       ...
4923     203     189       4  Directed
4924     203     190       1  Directed
4925     203     197       1  Directed
4926     203     201      12  Directed
4927     203     202       1  Directed

[4928 rows x 4 columns]
networks/2019-04_edges.csv
      Source  Target  Weight      Type
0          0     1.0       1  Directed
1          0    13.0      21  Directed
2          0    20.0       1  Directed
3          0    26.0       5  Directed
4          0    61